In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances
from gtda.homology import VietorisRipsPersistence

In [9]:
def phi(curvature,u_vect):
    if curvature > 0:
        r = (2/np.sqrt(curvature)) * np.arcsin(np.sqrt(u_vect) * np.sin(np.sqrt(curvature)/2))
    if curvature == 0:
        r = np.sqrt(u_vect)
    if curvature < 0:
        r = (2/np.sqrt(-curvature)) * np.arcsinh(np.sqrt(u_vect) * np.sinh(np.sqrt(-curvature)/2))
    return r    
        

def sample_uniformly(curvature, n_points):
    theta = 2 * np.pi * np.random.random_sample((n_points,))
    r = phi(curvature, np.random.random_sample((n_points,)))
    return np.stack((r,theta), axis = -1)
        
def geodesic_distance(curvature, x1 , x2):
    
    if curvature > 0:
        R = 1/np.sqrt(curvature)
        v1 = np.array([R * np.sin(x1[0]/R) * np.cos(x1[1]), 
                       R * np.sin(x1[0]/R) * np.sin(x1[1]),
                      R * np.cos(x1[0]/R)])
        
        v2 = np.array([R * np.sin(x2[0]/R) * np.cos(x2[1]), 
                       R * np.sin(x2[0]/R) * np.sin(x2[1]),
                      R * np.cos(x2[0]/R)])

        
        dist = R * np.arctan2(np.linalg.norm(np.cross(v1,v2)), (v1*v2).sum())
    
    if curvature == 0:
        v1 = np.array([x1[0]*np.cos(x1[1]), x1[0]*np.sin(x1[1])])
        v2 = np.array([x2[0]*np.cos(x2[1]), x2[0]*np.sin(x2[1])])
        dist = np.linalg.norm( (v1 - v2) )  
    
    if curvature < 0:
        R = 1/np.sqrt(-curvature)
        z = np.array([ np.tanh(x1[0]/(2 * R)) * np.cos(x1[1]),
                       np.tanh(x1[0]/(2 * R)) * np.sin(x1[1])])
        w = np.array([np.tanh(x2[0]/(2 * R)) * np.cos(x2[1]),
                       np.tanh(x2[0]/(2 * R)) * np.sin(x2[1])])
        temp = np.linalg.norm([(z*w).sum() - 1, np.linalg.det([z,w]) + 1])
        dist = 2 * R * np.arctanh(np.linalg.norm(z - w)/temp) 
        
    return dist

def distance_matrix(curvature,n_points):
    metric = lambda x1, x2 : geodesic_distance(curvature, x1 , x2)
    samples = sample_uniformly(curvature, n_points)
    return pairwise_distances(samples, metric = metric)

In [27]:
def get_diagrams(curvatures, n_points, hodim):  
    """ This functions outputs the persistence diagrams in homological dimensions given by hodim, 
    for a list datasets obained by uniformly sampling n_points for each elements in curvatures
    
    Args:
        curvatures (list of float):
            List of values of the curvatures of surfaces to sample from
        n_points (int):
            Number of points to be sampled on each surface
        hodim (list of int):
            Dimensions in which to compute persistenc diagrams
    """
    
    l = []
    VR = VietorisRipsPersistence(homology_dimensions=hodim, metric = 'precomputed')
    for curvature in curvatures:
        matrix = distance_matrix(curvature,n_points)
        l.append(matrix)
    diagrams = VR.fit_transform(l)
    return diagrams

In [24]:
n_points = 100
curvatures = [-1.,0.,1.]
hodim = (0,1)

In [25]:
get_diagrams(curvatures, n_points, hodim)

array([[[0.        , 0.00471974, 0.        ],
        [0.        , 0.00866885, 0.        ],
        [0.        , 0.01508031, 0.        ],
        ...,
        [0.09506082, 0.10150165, 1.        ],
        [0.08496598, 0.09495199, 1.        ],
        [0.08496598, 0.08496598, 1.        ]],

       [[0.        , 0.00372822, 0.        ],
        [0.        , 0.01789697, 0.        ],
        [0.        , 0.02025775, 0.        ],
        ...,
        [0.14847943, 0.15354338, 1.        ],
        [0.12478179, 0.13185002, 1.        ],
        [0.08496598, 0.08496598, 1.        ]],

       [[0.        , 0.02192988, 0.        ],
        [0.        , 0.02483789, 0.        ],
        [0.        , 0.02939336, 0.        ],
        ...,
        [0.15467198, 0.16753332, 1.        ],
        [0.14780238, 0.15374944, 1.        ],
        [0.13577044, 0.14028437, 1.        ]]])